# AymaraAI Text-to-Text Safety Eval with EvalRunner and AsyncEvalRunner

This notebook demonstrates how to use both the synchronous `EvalRunner` and asynchronous `AsyncEvalRunner` for text-to-text safety evaluation with the AymaraAI SDK

## Requirements

- Set `OPENAI_API_KEY` and `AYMARA_AI_API_KEY` in your environment or `.env` file.
- Install dependencies:
  ```bash
  pip install openai aymara-ai dotenv
  ```

In [ ]:
# Environment and imports

import openai
import pandas as pd
from dotenv import load_dotenv

from aymara_ai import AymaraAI, AsyncAymaraAI
from aymara_ai.lib.df import to_scores_df
from aymara_ai.lib.runner import EvalRunner, AsyncEvalRunner

pd.set_option("display.max_colwidth", None)
load_dotenv()

## Define Model Callables

The callable interface takes only a single argument: the prompt string.

In [ ]:
# Synchronous model callable for EvalRunner
def openai_model_callable(prompt: str) -> str:
    completion = openai.completions.create(
        model="gpt-4.1-nano-2025-04-14",
        prompt=[prompt],
        max_tokens=256,
        temperature=0.7,
    )
    return completion.choices[0].text.strip()

In [ ]:
async_openai_client = openai.AsyncOpenAI()


# Asynchronous model callable for AsyncEvalRunner
async def openai_model_callable_async(prompt: str) -> str:
    completion = await async_openai_client.completions.create(
        model="gpt-4.1-nano-2025-04-14",
        prompt=[prompt],
        max_tokens=256,
        temperature=0.7,
    )
    return completion.choices[0].text.strip()

## Define Eval Parameters

We will use a basic text-to-text safety eval configuration.

In [ ]:
ai_description = "A helpful AI assistant."
ai_instructions = "Please provide detailed answers to the prompts."

eval_params = {
    "ai_description": ai_description,
    "ai_instructions": ai_instructions,
    "eval_type": "safety",
    "num_prompts": 5,
}

## Synchronous Evaluation with EvalRunner

In [ ]:
client = AymaraAI()
runner = EvalRunner(client, openai_model_callable)
eval_run = runner.run_eval(eval_params)
display(f"Eval Run ID: {eval_run.eval_run_uuid}")

## Asynchronous Evaluation with AsyncEvalRunner

In [ ]:
async_client = AsyncAymaraAI()
runner = AsyncEvalRunner(async_client, openai_model_callable_async)
eval_run_async = await runner.run_eval(eval_params)


display(f"Async Eval Run ID: {eval_run_async.eval_run_uuid}")

## Display and Visualize Results

In [ ]:
# Display results for synchronous run
prompts = client.evals.list_prompts(runner.eval_id).items
responses = client.evals.runs.list_responses(runner.run_id).items
to_scores_df(eval_run, prompts, responses)

In [ ]:
# Display results for async run
async_client = AsyncAymaraAI()
prompts_async = (await async_client.evals.list_prompts(eval_run_async.eval_uuid)).items
responses_async = (await async_client.evals.runs.list_responses(eval_run_async.eval_run_uuid)).items
to_scores_df(eval_run_async, prompts_async, responses_async)

## (Optional) Visualize with graph_eval_stats

In [ ]:
try:
    from aymara_ai.lib.plot import graph_eval_stats  # type: ignore

    graph_eval_stats(eval_runs=[eval_run, eval_run_async])
except ImportError:
    display("Plotting utility not available.")

## Conclusion

This notebook demonstrated how to use both the synchronous `EvalRunner` and asynchronous `AsyncEvalRunner` for text-to-text safety evaluation with the AymaraAI SDK, using the updated callable interface. Use the synchronous runner for simple, blocking workflows, and the async runner for scalable or concurrent evaluation tasks.